In [1]:
import meshcat
from meshcat import Visualizer
import meshcat.geometry as mc_geom
import meshcat.transformations as mc_trans
# from mjcf_parser import parser
# from mjcf_parser import constants
from dm_control.mjcf import  parser
from dm_control.mjcf import constants
import mujoco 
import numpy as np


In [2]:
def _parse_mesh(_geom, _xml_model):
    return NULL
class MeshCatVisualizer(Visualizer):
    def __init__(self, xml_path, mj_model, mj_data) -> None:
        Visualizer.__init__(self)
        self._mj_data = mj_data
        self._mj_model = mj_model
        self._xml_path = xml_path
        self._xml_tree = parser.from_path(xml_path)
        self._mesh_dir = self._xml_tree.compiler.meshdir
        self._geom_names = []
        self._body_names = []
        self._add_body(self._xml_tree.worldbody, None)

    def _add_body(self, body, parent_body):
        body_idx = len(self._geom_names)
        if not parent_body:
            body_name = constants.WORLDBODY
        else:
            body_name = body.name if body.name else f"body{body_idx}"
        
        geoms = body.geom if hasattr(body, "geom") else []

        # _body = self[body_name]
        if body_name == constants.WORLDBODY:
            # TODO: parse the plan if it is in the environment
            pass
        else:
            for geom in geoms:
                _body = self[body_name]
                body_id = mujoco.mj_name2id(self._mj_model, mujoco.mjtObj.mjOBJ_BODY, body_name)
                body_pos = self._mj_data.xpos[body_id]
                body_quat = self._mj_data.xquat[body_id]
                tf = mc_trans.quaternion_matrix(body_quat)
                tf[:3,3]  = body_pos
                _body.set_transform(tf)
                self._body_names.append(body_name)
                if geom.type == constants.MESH:# or hasattr(geom, 'mesh'):
                    _geom = _body[geom.mesh.name]
                    _name = geom.mesh.file.get_vfs_filename().split('-')[0]
                    _ext = geom.mesh.file.extension
                    _xml_root = '/'.join(self._xml_path.split('/')[:-1])+'/.'
                    if _ext == '.stl':
                        geom_constr = mc_geom.StlMeshGeometry
                    elif _ext == '.obj':
                        geom_constr = mc_geom.ObjMeshGeometry

                    _geom.set_object(
                        geom_constr.from_file(
                            _xml_root + '/' + self._mesh_dir + _name + _ext
                        ),
                        mc_geom.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8)
                    )
                    geom_pos = np.zeros(3)
                    geom_quat = np.zeros(4)
                    geom_quat[0] = 1.0
                    if geom.pos is not None:
                        geom_pos[:] = geom.pos
                    if geom.quat is not None:
                        geom_quat[:] = geom.quat[:]
                    tf = mc_trans.quaternion_matrix(geom_quat)
                    tf[:3,3]  = geom_pos
                    _geom.set_transform(tf)

        # Recurse.
        for child_body in body.body:
            self._add_body(child_body, body)
    def render(self):
        for body_name in self._body_names:
            body_id = mujoco.mj_name2id(self._mj_model, mujoco.mjtObj.mjOBJ_BODY, body_name)
            body_xpos = self._mj_data.xpos[body_id]
            body_xmat = self._mj_data.xmat[body_id].reshape((3,3))            
            tf = np.eye(4)
            tf[:3,:3]   = body_xmat
            tf[:3,3]    = body_xpos
            self[body_name].set_transform(tf)


In [3]:
# xml_path = './a1/xml/a1.xml'
xml_path = './franka_emika_panda/scene_torque_actuators.xml'

model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)
visualizer = MeshCatVisualizer(xml_path, model, data)

ValueError: Error: joint has `range` but not `limited`. set the autolimits="true" compiler option, specify `limited` explicitly ("true" or "false"), or remove the `range` attribute.
Object name = joint1, id = 0, line = 0, column = -1

In [5]:
for _ in range(1000):
    mujoco.mj_step(model, data)

    visualizer.render()

KeyboardInterrupt: 

In [19]:
geom_name = visualizer._geom_names[0]
geom_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_MESH, geom_name)
geom_xpos = data.geom_xpos[geom_id]
geom_xmat = data.geom_xmat[geom_id].reshape((3,3))

In [34]:
mujoco.mjtObj.mjOBJ_GEOM

<mjtObj.mjOBJ_GEOM: 5>

In [23]:
data.geom_xpos.shape

(37, 3)

In [32]:
geom_name

'trunk'

In [28]:
mujoco.mjtObj.mjOBJ_MESH

<mjtObj.mjOBJ_MESH: 9>

In [30]:
model.geom_pos

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.83231838e-02,  4.39645123e-05,  5.01516492e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.01465656e-03, -1.26150927e-04,  1.86924988e-06],
       [ 0.00000000e+00, -5.50000000e-02,  0.00000000e+00],
       [-3.10750493e-03,  2.17985918e-02, -2.70516790e-02],
       [ 0.00000000e+00,  0.00000000e+00, -1.00000000e-01],
       [ 6.39596322e-03,  2.43462098e-05, -1.24936284e-01],
       [ 0.00000000e+00,  0.00000000e+00, -1.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.00000000e-01],
       [-8.01465656e-03,  1.26150927e-04, -1.86924988e-06],
       [ 0.00000000e+00,  5.50000000e-02,  0.00000000e+00],
       [-3.10780631e-03, -2.17962211e-02, -2.70684202e-02],
       [ 0.00000000e+00,  0.00000000e+00

In [24]:
len(visualizer._geom_names)

13

In [6]:
for _ in range(100000):
    mujoco.mj_step(model, data)
    visualizer.render()

KeyboardInterrupt: 

In [17]:
'/'.join(xml_path.split('/')[:-1])

'./a1/xml'

In [46]:
visualizer._xml_tree.worldbody.body[0].geom[0].mesh.file.get_vfs_filename().split('-')

['trunk', '06aa6e8f5faf130f61682e779415f4132196cdd6.stl']

In [41]:
visualizer._xml_tree.asset.mesh

MJCF Elements List: ['<mesh name="trunk" class="/" file="trunk-06aa6e8f5faf130f61682e779415f4132196cdd6.stl"/>', '<mesh name="hip" class="/" file="hip-80a0b0bdcc3bf251996c84e9ac5b7a53026ceca0.stl"/>', '<mesh name="thigh_mirror" class="/" file="thigh_mirror-82ee3d06a2feb71351e4274212ff53000e534b85.stl"/>', '<mesh name="calf" class="/" file="calf-2ff00292594a062d709752091469153dd8dfe7b5.stl"/>', '<mesh name="thigh" class="/" file="thigh-9aca36764f586ec5f074bedd039f4521bb2758cf.stl"/>']

In [11]:
visualizer._xml_tree.worldbody.body[0].geom[0].mesh.name

'trunk'

In [48]:
visualizer._xml_tree.worldbody.body[0].geom[0].mesh.name

'trunk'

In [10]:
visualizer._xml_tree.worldbody.body[0].name

'trunk'